# Data Mining

## Đề tài cuối kì: Dự đoán thời gian tắc đường tại các giao lộ

## Group 8


# 1. Giới thiệu

## 1.1 Giới thiệu

Tất cả chúng ta đều đã bị kẹt xe ở một cột đèn giao thông, chỉ được dành vài giây để đi qua một ngã tư, phía sau một đoàn người đi khác. Đó là vấn đề cần phải giải quyết, để có thể giúp các nhà lập kế hoạch thành phố và chính phủ dự đoán trước các điểm nóng về giao thông và giảm bớt căng thẳng khi dừng và đi của hàng triệu người khi tham gia giao thông.

Đầu tiên, ta cần import một số thư viện cần thiết vào đề tài.

In [ ]:
library(rsample)       
library(gbm)          
library(xgboost)     
library(caret)        
library(h2o)          
library(pdp)          
library(ggplot2)      
library(lime)         
library(stats)
library(dplyr)
library(gridExtra)
library(ISLR)
library(superml)

## 1.2 Dataset

Bộ dữ liệu cho cuộc thi này bao gồm thông tin tổng hợp về các phương tiện đã dừng và thời gian chờ ở giao lộ. Nhiệm vụ của bạn là dự đoán tình trạng tắc nghẽn, dựa trên thước đo tổng hợp về khoảng cách dừng xe và thời gian chờ, tại các giao lộ ở 4 thành phố lớn của Hoa Kỳ: Atlanta, Boston, Chicago & Philadelphia.

Tập dữ liệu này có 2 tập: train và test. Có số dòng như bên dưới:

In [ ]:
trainData <- read.csv(file = 'train.csv')
testData <- read.csv(file = 'test.csv')
nrow(trainData)
nrow(testData)

Một số dòng đầu như sau:

In [ ]:
head(trainData)

In [ ]:
head(testData)

## 1.3 Tiền xử lý dữ liệu

Chúng ta sẽ đi tìm sự khác biệt giữa tập train và tập test.

-   Tên các cột của tập train.

In [ ]:
print("train Col:")
print(names(trainData))

-   Tên các cột của tập test.

In [ ]:
print("test Col:")
print(names(testData))

Ta thấy rõ ràng rằng các value trong tập train có nhiều cột hơn tập test rất nhiều. Điều này chứng tỏ các col bị dư đều dùng để dự đoán. Tuy nhiên, theo yêu cầu competition này, ta chỉ cần dự đoán 6 biến. Gồm(TotalTimeStopped_p20, TotalTimeStopped_p50, TotalTimeStopped_p80, DistanceToFirstStop_p20, DistanceToFirstStop_p50, DistanceToFirstStop_p80)\
Do đó, ta sẽ loại bỏ các biến dư thừa còn lại.

In [ ]:
print(head(trainData[,names(trainData) %in% c("Path","EntryStreetName","EntryHeading","ExitStreetName","ExitHeading")]))

Với cột Path đã được mô tả ở 4 cột EntryStreetName, ExitStreetName, EntryHeading và ExitHeading nên ta sẽ bỏ không phân tích cột Path. Vả lại, đây là những biến category, chúng ta sẽ cần phải chuyển nó sang number trước khi tiến vào đào tạo mô hình.

Ở tạp train thì có nhiều cột tuy nhiên sang tập test thì chỉ còn 11 biến dự đoán nên nhóm lượt bỏ bớt những biến ở tập train có mà tập test không có. Mục đích để tăng hiệu suất mô hình.

In [ ]:
# We will remove some columns,
col<-c("IntersectionId","Latitude","Longitude","EntryStreetName","ExitStreetName","EntryHeading","ExitHeading","Hour","Weekend","Month","City","TotalTimeStopped_p20","DistanceToFirstStop_p20","TotalTimeStopped_p50","DistanceToFirstStop_p50","TotalTimeStopped_p80","DistanceToFirstStop_p80")
trainData<-trainData[,names(trainData) %in% col]
testData<-testData[,!(names(testData) %in% c("RowId","Path"))]

print("train Col:")
print(names(trainData))

In [ ]:
print("test Col:")
print(names(testData))

## 1.4 LabelEncoder

Bởi vì tập dữu liệu chứa các giá trị category, nên chúng ta cần xử lý những giá trị đó sang giá trị number để đào tạo mô hình. Dưới đây nhóm sử dụng hàm labelencoder trong thư viện `superml` cho các biến đã làm rõ ở trên.

In [ ]:
#Encoder Data


label <- LabelEncoder$new()

trainData$EntryStreetName <- label$fit_transform(trainData$EntryStreetName)

trainData$ExitStreetName <- label$fit_transform(trainData$ExitStreetName)

trainData$EntryHeading <- label$fit_transform(trainData$EntryHeading)

trainData$ExitHeading <- label$fit_transform(trainData$ExitHeading)

trainData$City <- label$fit_transform(trainData$City)


Tiếp theo để có cái nhìn chung hơn về mô hình đào tạo, nhóm sẽ chia tập train ra 7 phần để đào tạo, 3 phần để đánh giá mô hình.

In [ ]:
#We will divide trainData into train and val 
dx = sample(nrow(trainData),nrow(trainData)*0.7)
train<-trainData[dx,]
val<-trainData[-dx,]
nrow(train)
nrow(val)

Bước tiếp theo là tạo tập biến phản hồi Y, bởi vì tập train quá lớn (gần 600000 dòng) nên chúng ta sẽ giảm lại còn 10000 để có thể chạy được trên máy.

In [ ]:
#Now, we are having some problem with processing times, so we will only take some data from train and test
#So, if you can, you will modify num of train row and val row
object<-c('TotalTimeStopped_p20','TotalTimeStopped_p50','TotalTimeStopped_p80','DistanceToFirstStop_p20','DistanceToFirstStop_p50','DistanceToFirstStop_p80')

train_sp<-sample_n(train,10000)
val_sp<-sample_n(val,10000)

# 2. Mô hình đào tạo

## 2.1 Linear Regression

Chúng ta sẽ tạo một vòng lặp từ 1 tới 6 để mô hình chạy lần lượt các biến y, và xuất ra các kết quả. Các giá trị bên dưới đại diện cho RMSE của các biến y.

In [ ]:
#Try some models
#Linear
preFrame <- {}
valFrame<-{}

for(i in object[1:6]){
    rmc<-object[object!=i]
   
    train_20<-train_sp[,!(names(train_sp) %in% rmc)]
    val_20<-val_sp[,!(names(val_sp) %in% rmc)]
    ctrl <- trainControl(method = "cv",number = 7)
    
    lm<-switch(i,
        'TotalTimeStopped_p20'=train(TotalTimeStopped_p20 ~.,data = train_20,method = "lm",trControl = ctrl,),
        'DistanceToFirstStop_p20'=train(DistanceToFirstStop_p20~., data = train_20,method='lm',trControl = ctrl,),   
        'TotalTimeStopped_p50'=train(TotalTimeStopped_p50~., data = train_20,method='lm',trControl = ctrl,),   
        'DistanceToFirstStop_p50'=train(DistanceToFirstStop_p50~., data = train_20,method='lm',trControl = ctrl,),    
        'TotalTimeStopped_p80'=train(TotalTimeStopped_p80~., data = train_20,method='lm',trControl = ctrl,), 
        'DistanceToFirstStop_p80'=train(DistanceToFirstStop_p80~., data = train_20,method='lm',trControl = ctrl,)
      )
      x_val<-val_20[,!(names(val_20) %in% i)]
      y_val<-val_20[,(names(val_20) = i)]
      predicted <- predict(lm,x_val)  
      rmse<-RMSE(predicted,y_val)
      print(i)
      print(rmse)
    
      preFrame[[i]]<-predicted 
      valFrame[[i]]<-y_val
}
preFrame<-as.data.frame(preFrame)
valFrame<-as.data.frame(valFrame)
#print(preFrame)
#print(valFrame)

pre<-c(t(preFrame))
obs<-c(t(valFrame))

rmse<-RMSE(pre,obs)

## 2.2 K-nn model

Ở đây, nhóm sử dụng mô hình K-nn để dự đoán các giá trị như mô hình Linear. Mẫu ban đầu lấy là 10000, chia làm 7, Độ rộng để chạy k là 30, sau đó nó lấy kết quả tốt nhất:

In [ ]:
#knn
preFrame <- {}
valFrame<-{}

ctrl <- trainControl(method = "cv",number = 7)

for(i in object[1:6]){
    rmc<-object[object!=i]
    train_20<-train_sp[,!(names(train_sp) %in% rmc)]
    val_20<-train_sp[,!(names(train_sp) %in% rmc)]
    
    set.seed(1000)
    knn<-switch(i,
                'TotalTimeStopped_p20'=train(TotalTimeStopped_p20 ~ .,data = train_20,method = "knn", trControl = ctrl, preProcess = c("center","scale"), tuneLength = 30),
                'DistanceToFirstStop_p20'=train(DistanceToFirstStop_p20 ~ .,data = train_20,method="knn",trControl = ctrl, preProcess = c("center","scale"), tuneLength = 30),   
                'TotalTimeStopped_p50'=train(TotalTimeStopped_p50 ~ .,data = train_20,method = "knn",trControl = ctrl, preProcess = c("center","scale"), tuneLength = 30),
                'DistanceToFirstStop_p50'=train(DistanceToFirstStop_p50 ~ .,data=train_20,method="knn",trControl = ctrl, preProcess = c("center","scale"), tuneLength = 30),
                'TotalTimeStopped_p80'=train(TotalTimeStopped_p80 ~ .,data = train_20,method = "knn",trControl = ctrl, preProcess = c("center","scale"), tuneLength = 30),
                'DistanceToFirstStop_p80'=train(DistanceToFirstStop_p80 ~ .,data =train_20,method="knn",trControl = ctrl, preProcess = c("center","scale"), tuneLength = 30)
               )
    #print(knn)
    #print(plot(knn))
    
    x_val<-val_20[,!(names(val_20) %in% i)]
    y_val<-val_20[,(names(val_20) = i)]
    predicted <- predict(knn,x_val)  
    rmse<-RMSE(predicted,y_val)
    print(i)
    print(rmse)
    
    preFrame[[i]]<-predicted 
    valFrame[[i]]<-y_val
}
preFrame<-as.data.frame(preFrame)
valFrame<-as.data.frame(valFrame)
#print(preFrame)
#print(valFrame)

pre<-c(t(preFrame))
obs<-c(t(valFrame))

rmse<-RMSE(pre,obs)
#print("General target rmse")
#print(rmse)

Sau khi chạy xong mô hình K-nn, ta có kết quả như trên.

## 2.3 Random Forest model

Với mô hình Random Forest, nhóm sử dụng method `cross validation` với `number` = 7.

In [ ]:
#random forest
preFrame <- {}
valFrame<-{}

trControl = trainControl(method = "cv", number = 7, allowParallel = TRUE, verboseIter = FALSE)

for(i in object[1:6]){
    rmc<-object[object!=i]
    train_20<-train_sp[,!(names(train_sp) %in% rmc)]
    val_20<-train_sp[,!(names(train_sp) %in% rmc)]
    
    set.seed(1000)
    rf<-switch(i,
               'TotalTimeStopped_p20'= train(TotalTimeStopped_p20 ~.,data = train_20,method = "rf",trControl = trControl),
               'DistanceToFirstStop_p20'=train(DistanceToFirstStop_p20 ~., data = train_20,method = "rf",trControl = trControl),
               'TotalTimeStopped_p50'=train(TotalTimeStopped_p50 ~., data = train_20,method = "rf",trControl = trControl), 
               'DistanceToFirstStop_p50'=train(DistanceToFirstStop_p50 ~.,data = train_20,method = "rf",trControl = trControl),
               'TotalTimeStopped_p80'=train(TotalTimeStopped_p80 ~.,data = train_20,method = "rf",trControl = trControl),
               'DistanceToFirstStop_p80'=train(DistanceToFirstStop_p80 ~.,data = train_20,method = "rf", trControl = trControl)
              )
    x_val<-val_20[,!(names(val_20) %in% i)]
    y_val<-val_20[,(names(val_20) = i)]
    predicted <- predict(rf,x_val)  
    rmse<-RMSE(predicted,y_val)
    print(i)
    print(rmse)
    
    preFrame[[i]]<-predicted 
    valFrame[[i]]<-y_val
}
preFrame<-as.data.frame(preFrame)
valFrame<-as.data.frame(valFrame)
#print(preFrame)
#print(valFrame)

pre<-c(t(preFrame))
obs<-c(t(valFrame))

rmse<-RMSE(pre,obs)
#print("General target rmse")
#print(rmse)

Sau khi đào tạo mô hình hoàn tất, các giá trị được dự đoán như trên.

## 2.4 Gradient Boosting model

Cài đặt mặc định trong gbm có tỷ lệ học tập (learning rate) là 0,001. Đây là một tỷ lệ học tập rất nhỏ và thường yêu cầu một số lượng lớn cây để tìm ra MSE tối thiểu. Tuy nhiên, gbm sử dụng số lượng cây mặc định là 100, điều này là không đủ trong trường hợp này. Do đó, ta sẽ sử dụng lên đến 10.000 cây. Độ sâu mặc định của mỗi cây (depth) là 1, có nghĩa là chúng ta đang tập hợp một loạt các gốc cây. Cuối cùng, ta cũng sẽ sử dụng cv.folds để thực hiện xác thực chéo 5 lần. Mô hình mất khoảng 90 giây để chạy và kết quả cho thấy rằng hàm mất mát MSE của chúng tôi được giảm thiểu với 10.000 cây.

Ở đây, chúng ta sẽ sử dụng 1 biến phản hồi duy nhất `TotalTimeStopped_p20` để kiểm tra mô hình gbm này:

In [ ]:
# For gmb

#TotalTimeStopped_p20
object<-c('TotalTimeStopped_p50','TotalTimeStopped_p80','DistanceToFirstStop_p20','DistanceToFirstStop_p50','DistanceToFirstStop_p80')
train_20 <- train_sp[,!(names(train_sp) %in% object)]
val_20 <- val_sp[,!(names(val_sp) %in% object)]

set.seed(123)

# train GBM model
gbm.fit <- gbm(
  formula = TotalTimeStopped_p20 ~ .,
  distribution = "gaussian",
  data = train_20,
  n.trees = 1000,
  interaction.depth = 1,
  shrinkage = 0.001,
  cv.folds = 5,
  n.cores = NULL, # will use all cores by default
  verbose = FALSE
  )  



Ta đi tìm RMSE thông qua tính `sqrt` của MSE trong mô hình với các tham số trên.

In [ ]:
# get MSE and compute RMSE
sqrt(min(gbm.fit$cv.error))

# plot loss function as a result of n trees added to the ensemble
gbm.perf(gbm.fit, method = "cv")

RMSE của mô hình này hơn 6,7 giây cho biến phản hồi y `TotalTimeStopped_p20` nên ta cần phải thử các tham số khác để tìm ra mô hình dự đoán tốt nhất. Ta cần tạo một hyperparameter grid để áp dụng một lần nhiều giá trị khác nhau.

In [ ]:
# create hyperparameter grid
hyper_grid <- expand.grid(
  shrinkage = c(.01, .05, .1),
  interaction.depth = c(3, 5, 7),
  n.minobsinnode = c(5, 7, 10),
  bag.fraction = c(.65, .8, 1), 
  optimal_trees = 0,               # a place to dump results
  min_RMSE = 0                     # a place to dump results
)

# total number of combinations
nrow(hyper_grid)

In [ ]:
# randomize data
random_index <- sample(1:nrow(train_20), nrow(train_20))
random_train <- train_20[random_index, ]

# grid search 
for(i in 1:nrow(hyper_grid)) {
  
  # reproducibility
  set.seed(123)
  
  # train model
  gbm.tune <- gbm(
    formula = TotalTimeStopped_p20 ~ .,
    distribution = "gaussian",
    data = random_train,
    n.trees = 5000,
    interaction.depth = hyper_grid$interaction.depth[i],
    shrinkage = hyper_grid$shrinkage[i],
    n.minobsinnode = hyper_grid$n.minobsinnode[i],
    bag.fraction = hyper_grid$bag.fraction[i],
    train.fraction = .75,
    n.cores = NULL, # will use all cores by default
    verbose = FALSE
  )
  
  # add min training error and trees to grid
  hyper_grid$optimal_trees[i] <- which.min(gbm.tune$valid.error)
  hyper_grid$min_RMSE[i] <- sqrt(min(gbm.tune$valid.error))
}

hyper_grid %>% 
  dplyr::arrange(min_RMSE) %>%
  head(20)

Khi tìm thấy mô hình tốt nhất trong đây, ta đào tạo một mô hình với các thông số cụ thể đó. Và vì mô hình hội tụ ở 2619 cây nên chúng ta cần đào tạo một mô hình đã được xác thực chéo (để cung cấp ước tính sai số mạnh mẽ hơn) với 1000 cây.

In [ ]:
set.seed(123)

# train GBM model
gbm.fit.final <- gbm(
  formula = TotalTimeStopped_p20 ~ .,
  distribution = "gaussian",
  data = train_20,
  n.trees = 2619,
  interaction.depth = 5,
  shrinkage = 0.01,
  n.minobsinnode = 5,
  bag.fraction = 0.80, 
  train.fraction = 1,
  cv.folds = 5,
  #n.cores = NULL, # will use all cores by default
  verbose = FALSE
  ) 

In [ ]:
# get MSE and compute RMSE
sqrt(min(gbm.fit.final$cv.error))
a <- sqrt(min(gbm.fit.final$cv.error))
# plot loss function as a result of n trees added to the ensemble
gbm.perf(gbm.fit.final, method = "cv")

In [ ]:
#TotalTimeStopped_p50

object<-c('TotalTimeStopped_p20','TotalTimeStopped_p80','DistanceToFirstStop_p20','DistanceToFirstStop_p50','DistanceToFirstStop_p80')
train_20 <- train_sp[,!(names(train_sp) %in% object)]
val_20 <- val_sp[,!(names(val_sp) %in% object)]

# for reproducibility
set.seed(1000)

# train GBM model
gbm.fit_b <- gbm(
  formula = TotalTimeStopped_p50 ~ .,
  distribution = "gaussian",
  data = train_20,
  n.trees = 2619,
  interaction.depth = 5,
  shrinkage = 0.01,
  n.minobsinnode = 5,
  bag.fraction = 0.80, 
  train.fraction = 1,
  cv.folds = 5,
  n.cores = NULL, # will use all cores by default
  verbose = FALSE
  )  

# get MSE and compute RMSE
b <- sqrt(min(gbm.fit_b$cv.error))
bPre<-predict(gbm.fit_b,val_20)
bObs<-val_20

In [ ]:
#TotalTimeStopped_p80
object<-c('TotalTimeStopped_p20','TotalTimeStopped_p50','DistanceToFirstStop_p20','DistanceToFirstStop_p50','DistanceToFirstStop_p80')
train_20 <- train_sp[,!(names(train_sp) %in% object)]
val_20 <- val_sp[,!(names(val_sp) %in% object)]

# for reproducibility
set.seed(123)

# train GBM model
gbm.fit_c <- gbm(
  formula = TotalTimeStopped_p80 ~ .,
  distribution = "gaussian",
  data = train_20,
  n.trees = 2619,
  interaction.depth = 5,
  shrinkage = 0.01,
  n.minobsinnode = 5,
  bag.fraction = 0.80, 
  train.fraction = 1,
  cv.folds = 5,
  n.cores = NULL, # will use all cores by default
  verbose = FALSE
  )  

# get MSE and compute RMSE
c <- sqrt(min(gbm.fit_c$cv.error))
cPre<-predict(gbm.fit_c,val_20)
cObs<-val_20

In [ ]:
#DistanceToFirstStop_p20
object<-c('TotalTimeStopped_p20','TotalTimeStopped_p50','TotalTimeStopped_p80','DistanceToFirstStop_p50','DistanceToFirstStop_p80')
train_20 <- train_sp[,!(names(train_sp) %in% object)]
val_20 <- val_sp[,!(names(val_sp) %in% object)]

# for reproducibility
set.seed(123)

# train GBM model
gbm.fit_d <- gbm(
  formula = DistanceToFirstStop_p20 ~ .,
  distribution = "gaussian",
  data = train_20,
  n.trees = 2619,
  interaction.depth = 5,
  shrinkage = 0.01,
  n.minobsinnode = 5,
  bag.fraction = 0.80, 
  train.fraction = 1,
  cv.folds = 5,
  n.cores = NULL, # will use all cores by default
  verbose = FALSE
  )  

# get MSE and compute RMSE
d <- sqrt(min(gbm.fit_d$cv.error))
dPre<-predict(gbm.fit_d,val_20)
dObs<-val_20

In [ ]:
#DistanceToFirstStop_p50
object<-c('TotalTimeStopped_p20','TotalTimeStopped_p50','TotalTimeStopped_p80','DistanceToFirstStop_p20','DistanceToFirstStop_p80')
train_20 <- train_sp[,!(names(train_sp) %in% object)]
val_20 <- val_sp[,!(names(val_sp) %in% object)]
# for reproducibility
set.seed(123)

# train GBM model
gbm.fit_e <- gbm(
  formula = DistanceToFirstStop_p50 ~ .,
  distribution = "gaussian",
  data = train_20,
  n.trees = 2619,
  interaction.depth = 5,
  shrinkage = 0.01,
  n.minobsinnode = 5,
  bag.fraction = 0.80, 
  train.fraction = 1,
  cv.folds = 5,
  n.cores = NULL, # will use all cores by default
  verbose = FALSE
  )  

# get MSE and compute RMSE
e <- sqrt(min(gbm.fit_e$cv.error))
ePre<-predict(gbm.fit_e,val_20)
eObs<-val_20

In [ ]:
#DistanceToFirstStop_p80
object<-c('TotalTimeStopped_p20','TotalTimeStopped_p50','TotalTimeStopped_p80','DistanceToFirstStop_p20','DistanceToFirstStop_p50')
train_20 <- train_sp[,!(names(train_sp) %in% object)]
val_20 <- val_sp[,!(names(val_sp) %in% object)]

# for reproducibility
set.seed(123)

# train GBM model
gbm.fit_f <- gbm(
  formula = DistanceToFirstStop_p80 ~ .,
  distribution = "gaussian",
  data = train_20,
  n.trees = 2619,
  interaction.depth = 5,
  shrinkage = 0.01,
  n.minobsinnode = 5,
  bag.fraction = 0.80, 
  train.fraction = 1,
  cv.folds = 5,
  n.cores = NULL, # will use all cores by default
  verbose = FALSE
  )  

# get MSE and compute RMSE
f <- sqrt(min(gbm.fit_f$cv.error))
fPre<-predict(gbm.fit_f,val_20)
fObs<-val_20

In [ ]:
print("TotalTimeStopped_p20")
print(a)
print("DistanceToFirstStop_p20")
print(b)
print("TotalTimeStopped_p50")
print(c)
print("DistanceToFirstStop_p50")
print(d)
print("TotalTimeStopped_p80")
print(e)
print("DistanceToFirstStop_p80")
print(f)

Trên đây là kết quả khi traning với mô hình Gradient boosting, ta có thể thấy, nhìn chung, mô hình Gradient Boosting cho kết quả tốt hơn so với mô hình nhóm đã sử dụng trên.

Dưới đây, ta sẽ vẽ xem biểu đồ thể hiện loss function:

In [ ]:

# plot loss function as a result of n trees added to the ensemble
gbm.perf(gbm.fit_a, method = "cv")
gbm.perf(gbm.fit_b, method = "cv")
gbm.perf(gbm.fit_c, method = "cv")
gbm.perf(gbm.fit_d, method = "cv")
gbm.perf(gbm.fit_e, method = "cv")
gbm.perf(gbm.fit_f, method = "cv")

Để xem xét biến nào ảnh hưởng tới mô hình nhiều hơn, ta sẽ sử dụng biểu đồ phụ thuộc một phần với mô hình của `TotaltimeStoped_p20` với biến `Hour`.

In [ ]:
pdp <- gbm.fit_a %>%
  partial(pred.var = "Hour", n.trees = 1576, grid.resolution = 100, train = train_sp) %>%
  autoplot(rug = TRUE, train = train_sp)  +
  ggtitle("PDP")


gridExtra::grid.arrange(pdp, nrow = 1)

In [ ]:
submit<-read.csv(file = '../input/bigquery-geotab-intersection-congestion/sample_submission.csv')
submit$Target<-pre
write.csv(submit,'submission.csv')

In [ ]:
names(submit)
submit<-read.csv(file = './submit.csv')
nrow(submit)